# Geohash Sjoin Result Difference

In [2]:
import logging
import time
from datetime import datetime
from pathlib import Path
from shapely.geometry import Polygon, box
from polygon_geohasher.polygon_geohasher import polygon_to_geohashes, geohashes_to_polygon
import geohash
from functools import reduce
from math import ceil
import hvplot.pandas

import numpy as np
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
from distributed import LocalCluster, Client

In [3]:
cluster = LocalCluster(#silence_logs=logging.ERROR,
                       dashboard_address=':8790',
                       n_workers=4,
                       threads_per_worker=2,
                       memory_limit='3 GB')
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:36667 Dashboard: http://127.0.0.1:8790/status,Cluster Workers: 4 Cores: 8 Memory: 12.00 GB


In [4]:
base_path = Path('../')
contiguous_us_bounding_box = box(-124.848974, 24.396308, -66.885444, 49.384358)

In [5]:
# load contiguous us data
df = dd.read_parquet(base_path / 'data/contiguous_us_geohash4_sorted.parquet')
display(df.head(2))
len_df = len(df)
len_df

,latitude,longitude
geohash,,
9hre,24.591,-124.248
9hre,24.447,-124.443


113944489

In [6]:
%%time
# Save various size subsets of the zip code data
zips_1 = gpd.read_file(base_path / f'data/zip_codes/zips_1.geojson').loc[:, ['geometry']]
zips_10 = gpd.read_file(base_path / f'data/zip_codes/zips_10.geojson').loc[:, ['geometry']]
# zips_100 = gpd.read_file(base_path / f'data/zip_codes/zips_100.geojson').loc[:, ['geometry']]
# zips_1000 = gpd.read_file(base_path / f'data/zip_codes/zips_1000.geojson').loc[:, ['geometry']]
# zips_10000 = gpd.read_file(base_path / f'data/zip_codes/zips_10000.geojson').loc[:, ['geometry']]

CPU times: user 88.6 ms, sys: 15.7 ms, total: 104 ms
Wall time: 131 ms


In [7]:
%%time
total_points = len_df #len(df)
num_partitions = df.npartitions
geohash_precision = 4
num_polygons = []
time_sec = []
num_result_points = []
num_points = len(df.partitions[:num_partitions])

t00 = time.time()
for zip_gdf in [zips_1]:#, zips_10, zips_100, zips_1000, zips_10000]:
    num_polygons.append(len(zip_gdf))
    t0 = time.time()
    
    # get unique geohashes from data (could be precomputed and saved)
    unique_geohashes = df.index.unique().compute()
    
    # convert zip_codes to geohashes
    geohash_df = zip_gdf.geometry.apply(polygon_to_geohashes, 
                                                   precision=geohash_precision,
                                                   inner=False)#.apply(list)#.explode().to_frame()
    
    geohash_set = zip_geohashes = geohash_df.agg(lambda x: reduce(set.union, x))
    rdfs = []
#     for polygon_index, geohash_set in geohash_df.iteritems():
    zip_geohashes = list(geohash_set.intersection(unique_geohashes.values))  # filter out geohashes not in data 
    possible_interior_pts = df.loc[zip_geohashes]
    rdfs.append(possible_interior_pts)
#         rdfs.append(possible_interior_pts.map_partitions(spatial_join, zip_codes_gdf=zip_gdf.loc[polygon_index:polygon_index]))
    rdf = dd.concat(rdfs).compute()

    time_sec.append(time.time() - t0)
    num_result_points.append(len(rdf))
    print(f'num_polygons[-1]: {num_polygons[-1]}, time_sec[-1]: {time_sec[-1]:.0f} s')

num_polygons[-1]: 1, time_sec[-1]: 4 s
CPU times: user 1.23 s, sys: 97.7 ms, total: 1.33 s
Wall time: 6.55 s


In [8]:
# filter function
empty_df = pd.DataFrame([], columns=['latitude', 'longitude'])
def spatial_join(large_data_df, zip_codes_gdf):
    if large_data_df.empty:
        return empty_df
    crs = "epsg:4326"
    large_data_gdf = gpd.GeoDataFrame(large_data_df,
                                      geometry=gpd.points_from_xy(large_data_df.longitude,
                                                                  large_data_df.latitude),
                                      crs=crs)
    rdf = gpd.sjoin(large_data_gdf,
                    zip_codes_gdf,
                    how='inner',
                    op='within').drop(['index_right', 'geometry'], axis=1)
    if rdf.empty:
        return empty_df
    return rdf

In [9]:
%%time
total_points = len(df)
num_partitions = df.npartitions
geohash_precision = 4
num_polygons = []
time_sec = []
num_result_points = []
num_points = len(df.partitions[:num_partitions])

t00 = time.time()
for zip_gdf in [zips_1]:#, zips_10, zips_100, zips_1000]:#, zips_10000]:
    num_polygons.append(len(zip_gdf))
    t0 = time.time()
    
    # get unique geohashes from data (could be saved)
    unique_geohashes = df.index.unique().compute()
    
    # convert zip_codes to geohashes
    geohash_df = zip_gdf.geometry.apply(polygon_to_geohashes, 
                                                   precision=geohash_precision,
                                                   inner=False)#.apply(list)#.explode().to_frame()
    rdfs = []
    for polygon_index, geohash_set in geohash_df.iteritems():
        zip_geohashes = list(geohash_set.intersection(unique_geohashes.values))  # filter out geohashes not in data 
        possible_interior_pts = df.loc[zip_geohashes]
        rdfs.append(possible_interior_pts.map_partitions(spatial_join, zip_codes_gdf=zip_gdf.loc[polygon_index:polygon_index]))
    rdf2 = dd.concat(rdfs).compute()

    time_sec.append(time.time() - t0)
    num_result_points.append(len(rdf))
    print(f'num_polygons[-1]: {num_polygons[-1]}, time_sec[-1]: {time_sec[-1]:.0f} s')

num_polygons[-1]: 1, time_sec[-1]: 4 s
CPU times: user 1.91 s, sys: 131 ms, total: 2.04 s
Wall time: 10 s


In [10]:
polys = [geohashes_to_polygon([geohash]) for geohash in rdf.index.unique().values]

In [12]:
from holoviews.element.tiles import 

In [38]:
(gpd.GeoDataFrame(geometry=polys).hvplot(geo=True, color='red', alpha=0.1, tiles='CartoLight', label='Geohash Polygons') * \
zip_gdf.hvplot(geo=True, alpha=0.7, color='red', label='Zip Code Polygon') * \
rdf.hvplot.points('longitude', 'latitude', geo=True, label='Geohash Indexing Only', size=1.5, alpha=0.5, color='blue') * \
rdf2.hvplot.points('longitude', 'latitude', geo=True, color='cyan', size=1.5, alpha=0.2,
                              title='Comparison of Result Points based on Filtering Method',
                              xlabel='Longitude',
                              ylabel='Latitude',
                             label='With Spatial Join',
                            frame_height=300)
 
).opts(data_aspect=2)

:Overlay
   .Tiles.I                      :Tiles   [x,y]
   .Polygons.Geohash_Polygons    :Polygons   [Longitude,Latitude]
   .Polygons.Zip_Code_Polygon    :Polygons   [Longitude,Latitude]
   .Points.Geohash_Indexing_Only :Points   [longitude,latitude]
   .Points.With_Spatial_Join     :Points   [longitude,latitude]